In [31]:
import pandas as pd
import numpy as np
import re

In [2]:
acc = pd.read_csv('accused.csv.gz')
ilo = pd.read_csv('il-officers.csv.gz')
asw = pd.read_csv('all-sworn.csv.gz')

/Users/invinst/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
acc.head()

,CRID,Full.Name,Birth.Year,Gender,Race,Appointed.Date,Current.Unit,Current.Rank,Current.Star,Complaint.Category,Recommended.Finding,Recommended.Discipline,Final.Finding,Final.Discipline
0,258996,"BARRON, WILLIAM",1949.0,M,WHI,1978-02-27,018,SGT,NaN,01A-USE OF PROFANITY,NS,600.0,NS,600.0
1,258997,"C0NNOLLY, KIMBERLY",1965.0,F,BLK,1990-07-30,055,NaN,11026.0,01A-USE OF PROFANITY,UN,600.0,UN,600.0
2,258997,"KEENE, JOHN",1968.0,M,WHI,1999-03-08,153,PO,NaN,01A-USE OF PROFANITY,UN,600.0,UN,600.0
3,258998,"SLAVIN, SCOTT",1965.0,M,WHI,1991-11-18,145,SGT,807.0,10J-NEGLECT OF DUTY/CONDUCT UNBECOMING - ON DUTY,EX,600.0,EX,600.0
4,259001,"MARTINEZ, ANTONIO",1971.0,M,S,1996-11-04,701,PO,NaN,10U-INADEQUATE/FAILURE TO PROVIDE SERVICE,UN,600.0,UN,600.0


In [59]:
ilo.head()

,PTB.ID,Last.Name,First.Name,Middle.Name,Race,Department.Name,Position.Description,Appointed.Date,Resignation.Date,Birth.Year
0,65085238,Sandler,Mark,A,Caucasian/White,Iowa Interstate Railroad Police,"Law Enforcement, Full Time",1989-02-01,2000-12-20,1954
1,65012972,Button,Robert,NaN,Caucasian/White,Hebron Police Dept,"Law Enforcement, Part Time",1977-04-07,2010-01-01,1941
2,65017919,Collins,Eric,R,Caucasian/White,Hebron Police Dept,"Law Enforcement, Part Time",1994-10-24,2010-01-01,1972
3,65063914,McNabb,Joel,R,Caucasian/White,Hebron Police Dept,"Law Enforcement, Part Time",1992-05-18,2010-01-01,1971
4,65008259,Bockelmann,Daniel,A,Caucasian/White,Johnsburg Police Dept,"Law Enforcement, Full Time",1992-06-09,NaN,1970


In [60]:
asw.head()

,Last.Name,First.Name,Middle.Initial,Gender,Race,Birth.Year,Appointed.Date
0,AARON,JEFFERY,M,M,WHITE,1971.0,2005-09-26
1,AARON,KARINA,NaN,F,WHITE HISPANIC,1980.0,2005-09-26
2,ABATE,DANIEL,P,M,WHITE,1942.0,1970-06-15
3,ABBATE,ANTHONY,G,M,WHITE,1968.0,1994-12-05
4,ABBATE,CARMEL,G,M,WHITE,1942.0,1969-01-06


In [4]:
gender_df = pd.read_csv('../hand/gender_dictionary.csv')
race_df = pd.read_csv('../hand/race_dictionary.csv')
gender_dict = dict(zip(gender_df.Original, gender_df.Standard))
race_dict = dict(zip(race_df.Original, race_df.Standard))

In [221]:
def list_diff(l1, l2):
    '''returns list after taking set difference of two lists'''
    return list(set(l1) - set(l2))

def intersect(l1, l2):
    '''returns list after taking set intersection of two lists'''
    return list(set(l1) & set(l2))

def clean_int(x, na_value=np.nan):
    '''returns an integer from an object if possible,
       else returns an na_value
    '''
    if isinstance(x, str):
        # Check to see if it the string may be a float
        if re.search('^[0-9,.]*$', x):
            return int(float(x))
        # If not, return na_value
        else:
            return na_value
    elif np.isfinite(x):
        return int(float(x))
    else:
        return na_value
    
def standardize_gender(x):
    '''returns a standardized gender string
       by passing input string into gender reference dictionary
    '''
    # Ensure input gender (x) is string
    if isinstance(x, str):
        x = x.upper()   # Change x to uppercase
        # Check if x is already standardized
        if x in gender_dict.values():
            return x
        # If not, pass x into gender dictionary for standardization
        else:
            return gender_dict[x]
    # If not a string return 'NAN'
    else:
        return gender_dict['NAN']

def standardize_race(x):
    '''returns a standardized race string
       by passing input string into race reference dictionary
    '''
    # Ensure input race (x) is string
    if isinstance(x, str):
        x = x.upper()   # Change x to uppercase
        # Check if x is already standardized
        if x in race_dict.values():
            return x
        # If not, pass x in race dictionary for standardization
        else:
            return race_dict[x]
    # If not a string return 'NAN'
    else:
        return race_dict['NAN']
    
def clean_date_df(df):
    '''returns pandas dataframe of cleaned date and time columns
       splits datetime columns into date and time,
       ensures any errors are returned as null.
    '''
    # Store column names
    # All columns must end with .Date or .Datetime
    df_cols = df.columns.values  # Store column names
    dt_df = pd.DataFrame()  # Create empty dataframe

    # Iterate over stored date(time) columns
    for col in df_cols:
        # Store column suffix as list, removing .Date(time) ending
        # 'Org.Hire.Datetime' -> ['Org', 'Hire']
        col_suffix = col.split('.')[:-1]
        # Try to convert column to datetime
        # And create column in dt_df ending with .Date
        try:
            dt_df['.'.join(col_suffix + ["Date"])] = \
                pd.to_datetime(df[col], errors='raise').dt.date
        # If there were errors, notify the user,
        # And repeat above but with coercing errors to NaT
        except:
            print('Some errors in {}. Returned as NaT.'.format(col))
            dt_df['.'.join(col_suffix + ["Date"])] = \
                pd.to_datetime(df[col], errors='coerce').dt.date

        # If time is in column, repeat above
        # Except convert to time not date
        if 'time' in col:
            try:
                dt_df['.'.join(col_suffix + ["Time"])] = \
                    pd.to_datetime(df[col]).dt.time
            except:
                print('Some errors in {}. Returned as NaT.'.format(col))
                dt_df['.'.join(col_suffix + ["Time"])] = \
                    pd.to_datetime(df[col], errors='coerce').dt.date

    # EX: df columns = ['Org.Hire.Datetime', 'Start.Date']
    #     dt_df columns = ['Org.Hire.Date', 'Org.Hire.Time', 'Start.Date']
    return dt_df

def strip_name(x):
    '''returns string after
       removing any redundant whitespace or punctuation from string
    '''
    x = re.sub(r'[^\w\s]', '', x)
    return ' '.join(x.split())


def split_full_names(names, ln='Last.Name', fn='First.Name'):
    '''returns pandas dataframe of last and first name columns
       made from splitting input pandas series of full names
    '''
    # Fill NaN names with ',' for ease of use
    names = names.fillna(",")
    # Fill names that do not contain letter ('-') as ',' for ease of use
    names = names.map(lambda x: x if re.search('[a-zA-Z]', x) else ",")
    # Split names after last comma into list of length = 2
    # Ex: 'Jones, JR, Bob' -> ['Jones, JR', 'Bob']
    names = names.map(lambda x: x.rsplit(',', 1))
    # Turn series of lists into dataframe
    # with last and first name columns
    names = pd.DataFrame(names.values.tolist(),
                         columns=[ln, fn])
    return names

def extract_suffix_name(x):
    '''returns the suffix name in a given string'''
    suffixes = ('II', 'III', 'IV', 'JR', 'SR')
    # Split input name (x) by spaces and look for suffix in name pieces
    suffix = [w for w in x.split(" ") if w in suffixes]
    # If any suffix found, return it, else return empty string
    return suffix[0] if suffix else ""


def extract_middle_initial(x, mi_pattern):
    '''returns the middle initial in a given string'''
    mi = re.search(mi_pattern, x)
    mi = mi.group() if mi and len(x) > 3 else ''
    return mi

def split_name(x, mi_pattern):
    x = strip_name(x)
    mi = extract_middle_initial(x, mi_pattern)
    if mi:
        x = x.replace(mi, ' ')
        mi = mi.replace(' ', '')
    suff = extract_suffix_names(x)
    if suff:
        x = x.replace(suff, '')
        suff = suff.replace(' ', '')
    if not x:
        x = suff
        suff = ''
    x = x.replace(' ', '')
    return [x, mi, suff]
        
def clean_name_col(col, col_name):
    if col_name == 'First.Name':
        mi_pattern = '(^[A-Z]\s)|(\s[A-Z])\s|\s[A-Z]$'
    elif col_name == 'Last.Name':
        mi_pattern = '(\s[A-Z])\s|\s[A-Z]$'
    elif col_name == 'Middle.Name':
        col = col.map(strip_name)
        col = col.map(str.upper)
        return pd.DataFrame([['', '']
                             if not (isinstance(x, str) and
                                     x)
                             else [x,x[0]]
                             for x in col],
                           columns=[col_name, 'Middle.Initial'])
    else:
        raise Exception('Uh.. what sort of names?')
    col = col.map(strip_name)
    col = col.map(str.upper)
    return pd.DataFrame([split_name(x, mi_pattern) for x in col],
                        columns=[col_name, 'Middle.Initial', 'Suffix.Name'])
    
def clean_names(df):
    '''returns pandas dataframe of cleaned name columns'''
    df_cols = df.columns.values  # Store column names
    # If names are Full.Names, split them into last and first names
    # Then remove suffix and middle initial respectively
    if 'Full.Name' in df_cols:
        name_df = split_full_names(df['Full.Name'])
    else:
        name_df = df.fillna("")  # Fill NAs with empty strings
    
    fn = name_df['First.Name']
    fn_df = clean_name_col(fn, 'First.Name')
    ln = name_df['Last.Name']
    ln_df = clean_name_col(ln, 'Last.Name')
    
    mi_df = fn_df[['Middle.Initial']].join(ln_df['Middle.Initial'])
    sn_df = fn_df[['Suffix.Name']].join(ln_df['Suffix.Name'])
    cleaned_df = fn_df[['First.Name']].join(ln_df['Last.Name'])
    if 'Middle.Name' in df_cols:
        mn_df = clean_name_col(df['Middle.Name'], 'Middle.Name')
        cleaned_df = cleaned_df.join(mn_df['Middle.Name'])
        mi_df = mi_df.join(mn_df['Middle.Initial'])
    
    
    '''# Join last (and suffix) and first (and middle initial) dataframes
    name_df = last.join(first)

    # If middle initial column already existed in dataframe
    if 'Middle.Initial' in df_cols:
        # Print any conflicts between original middle initial and generated
        print('Middle Initial Conflicts:',
              name_df.ix[(df['Middle.Initial'] != '') &
                         (name_df['Middle.Initial'] != '')])
        # Replace generated middle initials with original middle initials
        # at locations with non-empty middle initials in origin dataframe
        name_df.ix[df['Middle.Initial'] != '', 'Middle.Initial'] = \
            df.ix[df['Middle.Initial'] != '', 'Middle.Initial']

    # If suffix column already existed in dataframe
    if 'Suffix.Name' in df_cols:
        # Print any conflicts between original suffix and generated
        print('Suffix Name Conflicts:',
              name_df.ix[(df['Suffix.Name'] != '') &
                         (name_df['Suffix.Name'] != '')])
        # Replace generated suffixes with original suffixes
        # at locations with non-empty suffixes in origin dataframe
        name_df.ix[df['Suffix.Name'] != '', 'Suffix.Name'] = \
            df.ix[df['Suffix.Name'] != '', 'Suffix.Name']
    # Replace any black spaces with empty string
    name_df[name_df == ' '] = '''
    return name_df

In [222]:
name_cols = ['Full.Name','First.Name', 'Last.Name', 
             'Middle.Initial', 'Middle.Name',
             'Suffix.Name']

In [223]:
ilon = ilo[['Last.Name','First.Name', 'Middle.Name']]
ln = clean_name_col(ilon['Last.Name'], 'Last.Name')
fn = clean_name_col(ilon['First.Name'], 'First.Name')

In [224]:
fn[fn['Middle.Initial']!='']

,First.Name,Middle.Initial,Suffix.Name
6094,GILBERT,G,
16982,ROBERT,J,
32046,THOMAS,R,
36890,MAURICE,H,
39185,RALPH,A,
149071,LLOYD,G,JR


In [225]:
i = 'LLOYD G JR'
i = strip_names(i)
i
mi = re.search('(^[A-Z]\s)|(\s[A-Z])\s|\s[A-Z]$', i)
mi.group()

' G '

In [227]:
col = ilo['Middle.Name']
[['', '']
 if not (isinstance(x, str) and x)
 else [x,x[0]]
 for x in col]

[['A', 'A'],
 ['', ''],
 ['R', 'R'],
 ['R', 'R'],
 ['A', 'A'],
 ['C', 'C'],
 ['C', 'C'],
 ['A', 'A'],
 ['L', 'L'],
 ['D', 'D'],
 ['J', 'J'],
 ['R', 'R'],
 ['L', 'L'],
 ['A', 'A'],
 ['', ''],
 ['L', 'L'],
 ['E', 'E'],
 ['D', 'D'],
 ['Dean', 'D'],
 ['T', 'T'],
 ['R', 'R'],
 ['', ''],
 ['J', 'J'],
 ['M', 'M'],
 ['A', 'A'],
 ['', ''],
 ['E', 'E'],
 ['J', 'J'],
 ['S', 'S'],
 ['B', 'B'],
 ['M', 'M'],
 ['J', 'J'],
 ['E', 'E'],
 ['P', 'P'],
 ['G', 'G'],
 ['K', 'K'],
 ['A', 'A'],
 ['A', 'A'],
 ['J', 'J'],
 ['F', 'F'],
 ['P', 'P'],
 ['M', 'M'],
 ['W', 'W'],
 ['W', 'W'],
 ['L', 'L'],
 ['E', 'E'],
 ['M', 'M'],
 ['C', 'C'],
 ['M', 'M'],
 ['W', 'W'],
 ['J', 'J'],
 ['', ''],
 ['C', 'C'],
 ['M', 'M'],
 ['A', 'A'],
 ['R', 'R'],
 ['J', 'J'],
 ['E', 'E'],
 ['A', 'A'],
 ['A', 'A'],
 ['N', 'N'],
 ['M', 'M'],
 ['A', 'A'],
 ['A', 'A'],
 ['D', 'D'],
 ['Edwin', 'E'],
 ['Allen', 'A'],
 ['L', 'L'],
 ['Guy', 'G'],
 ['Lawrence', 'L'],
 ['Jerry', 'J'],
 ['Todd', 'T'],
 ['Antoine', 'A'],
 ['Thomas', 'T'],
 ['Berlang

In [228]:
df = ilo[intersect(ilo.columns, name_cols)]

In [233]:
df_cols = df.columns.values  # Store column names
    # If names are Full.Names, split them into last and first names
    # Then remove suffix and middle initial respectively
if 'Full.Name' in df_cols:
    name_df = split_full_names(df['Full.Name'])
else:
    name_df = df.fillna("")  # Fill NAs with empty strings

In [287]:
fn = name_df['First.Name']
fn_df = clean_name_col(fn, 'First.Name')
ln = name_df['Last.Name']
ln_df = clean_name_col(ln, 'Last.Name')
cleaned_df = fn_df[['First.Name']].join(ln_df['Last.Name'])

In [288]:
mi_list = []
sn_list = []
mi_list.append(fn_df['Middle.Initial'])
mi_list.append(ln_df['Middle.Initial'])
sn_list.append(fn_df['Suffix.Name'])
sn_list.append(ln_df['Suffix.Name'])

In [289]:
if 'Middle.Name' in df_cols:
    mn_df = clean_name_col(name_df['Middle.Name'], 'Middle.Name')
    cleaned_df = cleaned_df.join(mn_df['Middle.Name'])
    mi_list.append(mn_df['Middle.Initial'])

In [290]:
mis = pd.concat(mi_list, axis=1)
sns = pd.concat(sn_list, axis=1)

In [297]:
for i,f, l, m in mis.itertuples():
    if l:
        print(i,f,l, m)

94564  O A
150649  V C
151716  O 
153695  V J


In [296]:
df.ix[153695]

Last.Name      Mose V
First.Name      James
Middle.Name         J
Name: 153695, dtype: object

In [298]:
for i,f, l in sns.itertuples():
    if l:
        print(i,f,l)

1644  JR
1943  JR
2140  JR
5104  JR
6597  JR
7714  JR
8965  JR
10866  JR
10867  III
10929  SR
11067  JR
11084  SR
11128  JR
11487  JR
11511  JR
11553  SR
11612  JR
11629  JR
11634  JR
11650  JR
11777  JR
11810  JR
11846  JR
11850  JR
11964  JR
12136  III
12160  JR
12194  JR
12217  JR
12219  JR
12221  JR
12224  JR
12326  JR
12435  III
12446  III
12458  III
12521  JR
12551  JR
12633  SR
12817  JR
12865  JR
12870  JR
12909  JR
13066  JR
13171  JR
13184  II
13188  JR
13431  JR
13435  JR
13477  III
13483  JR
13547  JR
13622  JR
13677  JR
13694  II
13705  JR
13807  JR
13868  III
13916  III
13966  JR
14087  JR
14160  JR
14220  JR
14230  IV
14264  JR
14270  JR
14353  II
14472  SR
14504  SR
14507  JR
14545  JR
14549  III
14766  JR
14802  JR
14830  JR
14856  JR
14874  JR
14928  JR
14934  JR
14945  JR
14997  JR
15136  JR
15171  JR
15197  JR
15312  JR
15347  JR
15396  JR
15453  JR
15531  JR
15590  JR
15612  JR
15613  JR
15632  JR
15728  JR
15741  JR
15915  III
16002  JR
16013  IV
16030  JR
16110  